In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(
    "/home/aca/Documents/S6_Lab/Projects/libraryDataAnalysis/pattern_mining/book_issue_with_book_id.csv")
min_date = min(df['issue_date']) # 2016-05-06
max_date = max(df['issue_date']) # 2022-06-08
# print(max_date)
print(min_date)

2016-05-06


In [3]:
# Filter records. Minimise headache.

# TODO: Add more filters

# Filters

# start_date = '2021-01-01'
# end_date = '2021-12-31'

start_date = min_date
end_date = max_date

min_supp_count = 10 # Number of transactions that must support the association rule
asso_rule_metric = "confidence"
min_threshold = 0.5

In [4]:
# Applying filters 
fdf = df[(df['issue_date'] >= start_date) & (df['issue_date'] <= end_date)]
total_transactions = (fdf.shape)[0]  # 1780 rows
min_support = min_supp_count/total_transactions
total_transactions

40329

In [5]:
fdf

,id,acc_no,title,issue_date,author,publisher,type,department,batch,book_id
0,NT023,531,PRINCIPLES OF PHYSICAL CHEMISTRY,2016-08-12,PURI&SHARMA&PATHANIA,SHOBANLAL PUB,staff,NaN,0,3466
1,NT023,706,TEXT BOOK OF ORGANIC CHEMISTRY,2016-08-01,"TEWARI,KA&VISHNOI,N K&MEHROTRA,S N",VIKAS PUBLISHING HOUSE,staff,NaN,0,4029
2,NT023,831,PHYSICAL CHEMISTRY,2016-08-12,"CASTELLAN,GILBERT W",NAROSA PUBLISHING HOUSE,staff,NaN,0,3303
3,NT023,13478,HARRY POTTER AND THE HALF-BLOOD PRINCE,2016-09-23,"ROWLING, J K",BLOOMSBURY,staff,NaN,0,1990
4,NT023,13498,SHERLOCK HOLMES SAMPOORNA KRUTHIKAL VOL.2,2016-11-04,"CONAN DOYLE, SIR ARTHUR",D C BOOKS,staff,NaN,0,3762
...,...,...,...,...,...,...,...,...,...,...
40324,20B671,7943,FUNDAMENTALS OF LOGIC DESIGN,2022-03-04,"ROTH,CHARLES H",CENGAGE LEARNING INDIA PVT LTD.,student,ME,2020,1819
40325,20B818,18366,TEXTBOOK OF STRENGTH OF MATERIALS,2022-03-18,"BANSAL,R K",LAXMI PUBLICATION,student,ME,2020,4051
40326,457316,131,3000 SOLVED PROBLEMS IN PHYSICS,2021-03-18,"HALPERN,ALVIN",McGRAW-HILL,staff,MECHANICAL,0,7
40327,457316,12772,ENGINEERING OPTIMIZATION,2021-03-18,"RAVINDRAN,A & RAGSDELL,K M & REKLAITIS",WILEY INDIA,staff,MECHANICAL,0,1564


In [6]:
# Finding average support

# Number of times each book was issued. 
fdf1 = fdf.groupby(['book_id'], as_index=False)['title'].count()

In [8]:
# Measures of central tendency
mean = fdf1['title'].mean()
mode = fdf1['title'].mode()
median = fdf1['title'].median()
sum_ = fdf1['title'].sum()
max_ = max(fdf1['title'])
min_ = min(fdf1['title'])

print(f"""
Mean = {mean}
Mode = {mode.tolist()}
Median = {median}
Sum = {sum_}
Max = {max_}
Min = {min_}
""")



Mean = 18.398266423357665
Mode = [1]
Median = 5.0
Sum = 40329
Max = 728
Min = 1



In [7]:
# Create transaction table. Each element implies books that were issued together.
# Took 35 seconds to finish 
items = []
unq_dates = fdf['issue_date'].drop_duplicates().tolist()
for i in unq_dates:
    temp = fdf[fdf['issue_date'] == i]
    unq_ids = temp['id'].drop_duplicates().tolist()
    for j in unq_ids:
        items.append(temp[temp['id'] == j]['book_id'].tolist())
print(len(items)) # Number of entries in transaction table 

29500


## Association Rule Mining

In [8]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules


In [9]:
te = TransactionEncoder()
te_ary = te.fit(items).transform(items)

In [10]:
df2 = pd.DataFrame(te_ary, columns=te.columns_)

In [13]:
# Store dataframe for future purpose. This can be used by both fp growth and apriori
# df2.to_pickle("te_ary_all.pkl")
# df2.to_html("test.html")
# to read pickle
# df2.read_pickle("te_ary_all.pkl")

### Apriori

In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(df2, min_support = min_support, use_colnames=True)

# frequent_itemsets
# Causes memory error with large data sets : Unable to allocate 23.4 GiB for an array with shape (425503, 2, 29500) and data type bool

In [ ]:
res1 = association_rules(frequent_itemsets, metric = asso_rule_metric, min_threshold=min_threshold)
res1.sort_values('confidence',ascending=False)
res1

### FP Growth

In [11]:
#Importing Libraries
from mlxtend.frequent_patterns import fpgrowth

#running the fpgrowth algorithm
res = fpgrowth(df2, min_support=min_support, use_colnames=True)

# printing top 10
# res.head(10)


In [17]:

res1 = association_rules(res, metric=asso_rule_metric, min_threshold=0.2)
res2 = res1.sort_values(by=['confidence', 'lift'], ascending=False)
res2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10,"(850, 2868)",(4458),0.000475,0.006441,0.000373,0.785714,121.992481,0.000370,4.636610
69,(961),(953),0.000373,0.024644,0.000271,0.727273,29.511067,0.000262,3.576305
55,(4334),(3633),0.000542,0.000983,0.000373,0.687500,699.353448,0.000372,3.196854
42,(281),(1651),0.001051,0.002169,0.000712,0.677419,312.247984,0.000710,3.093275
61,"(3475, 2620)",(2299),0.000475,0.001661,0.000305,0.642857,387.026239,0.000304,2.795349
...,...,...,...,...,...,...,...,...,...
17,"(4458, 2868)",(849),0.001492,0.008034,0.000305,0.204545,25.460299,0.000293,1.247043
3,(320),(319),0.002339,0.002373,0.000475,0.202899,85.507246,0.000469,1.251569
5,(320),(953),0.002339,0.024644,0.000475,0.202899,8.233160,0.000417,1.223628
25,(2899),(658),0.004034,0.003051,0.000814,0.201681,66.106443,0.000801,1.248810


In [ ]:
for i in res1['antecedents']:
    print(type(i))


In [20]:
for index, row in res1.iterrows():
    print(type(row["antecedents"]))


<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
<class 'frozenset'>
